            # PhysAE – optimisation d'hyperparamètres avec Optuna

            Ce carnet configure une étude Optuna ciblant la phase B1 et analyse les
            résultats pour mettre à jour les fichiers YAML.
            


            ## 1. Préparation

            Nous réutilisons le constructeur de données pour garantir que la
            distribution de validation reste identique à celle des stages
            d'entraînement.
            


In [ ]:
            from pathlib import Path
            import optuna

            from physae.factory import build_data_and_model
            from physae.optimization import optimise_stage

            STUDY_DIR = Path('artifacts/optuna')
            STUDY_DIR.mkdir(parents=True, exist_ok=True)
            model, train_loader, val_loader = build_data_and_model(config_overrides={'n_train': 4096, 'n_val': 512, 'batch_size': 12})
            print('Prêt à lancer une étude Optuna...')
            


            ## 2. Lancement de l'étude

            Nous limitons le nombre d'essais pour rester compatible avec un
            environnement CPU. Les `sampler` et `pruner` par défaut peuvent être
            remplacés si besoin.
            


In [ ]:
            study = optimise_stage(
                'B1',
                n_trials=10,
                metric='val_loss',
                base_model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                study_name='physae-b1-demo',
                storage=f"sqlite:///{(STUDY_DIR/'b1_demo.sqlite').as_posix()}",
                load_if_exists=True,
            )
            study.best_params
            


            ## 3. Export des meilleurs réglages

            Une fois satisfait des performances, on sauvegarde les hyperparamètres
            optimalement trouvés dans un fichier YAML utilisable lors de prochaines
            sessions.
            


In [ ]:
            import yaml

            best_cfg = {
                'stage': 'B1',
                'overrides': study.best_params,
            }
            output_path = STUDY_DIR / 'b1_best_params.yaml'
            output_path.write_text(yaml.safe_dump(best_cfg, sort_keys=False))
            print('Hyperparamètres sauvegardés dans', output_path)
            
